# Partitioning

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
clean_9000 = 'result_clean_pdf_9000_files.csv'
clean_109 = 'result_clean_pdf_109_embed_video.csv'
malw_173 = 'result_malware_pdf_cve_sorted_173_files.csv'
malw_10982 = 'result_malware_pdf_pre_04-2011_10982_files.csv'
df_clean_9000 = pd.read_csv(clean_9000, sep=',')
df_clean_109 = pd.read_csv(clean_109, sep=',')
df_malw_173 = pd.read_csv(malw_173, sep=',')
df_malw_10982 = pd.read_csv(malw_10982, sep=',')

In [3]:
df_clean = pd.concat([df_clean_9000, df_clean_109], axis=0, ignore_index=True)
df_malw = pd.concat([df_malw_10982, df_malw_173], axis=0, ignore_index=True)

df_clean["malware"] = np.zeros(df_clean.shape[0], dtype='bool')
df_malw["malware"] = np.ones(df_malw.shape[0], dtype='bool')

In [4]:
df_tot = pd.concat([df_clean, df_malw], axis=0, ignore_index=True)
X_tot = df_tot.iloc[:,:-1]
y_tot = df_tot.iloc[:,-1:]
X_train, X_test, y_train, y_test= train_test_split(X_tot, y_tot,
                                                    test_size= 0.2,
                                                    shuffle= True, #shuffle the data to avoid bias
                                                    stratify=df_tot['malware'],
                                                    random_state= 0)
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train.values.ravel())
np.save('y_test.npy', y_test.values.ravel())
np.save('X_tot.npy', X_tot)
df_tot.to_pickle('df_tot.pandas')

In [36]:
df_cic = pd.read_csv('PDFMalware2022.csv', sep=',')
df_cic.rename(columns={'Class':'malware'}, inplace=True)
df_cic.loc[df_cic['malware'] == 'Malicious', 'malware'] = True
df_cic.loc[df_cic['malware'] == 'Benign', 'malware'] = True
df_cic = df_cic.drop(columns=['Fine name', 'header', 'text'])
X_tot = df_tot.iloc[:,:-1]
y_tot = df_tot.iloc[:,-1:]
X_train, X_test, y_train, y_test= train_test_split(X_tot, y_tot,
                                                    test_size= 0.2,
                                                    shuffle= True, #shuffle the data to avoid bias
                                                    stratify=df_tot['malware'],
                                                    random_state= 0)
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
y_test_predicted = knn_clf.predict(X_test).reshape(-1,1)
print('Accuracy on testing: %f%%' % (metrics.accuracy_score(y_test, y_test_predicted)*100))
y_train_predicted = knn_clf.predict(X_train).reshape(-1,1)
print('Accuracy on training: %f%%' % (metrics.accuracy_score(y_train, y_train_predicted)*100))

/Users/mattiarosso/opt/anaconda3/envs/malis/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Accuracy on testing: 99.529936%
Accuracy on training: 99.690728%


In [38]:
y_train

,malware
17252,True
1052,False
13389,True
6090,False
9993,True
...,...
10953,True
5817,False
11897,True
10557,True
